# data_utils

> Utilities for dataset generation and tokenization

In [ ]:
#| default_exp data_utils

In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

In [ ]:
#| hide

from nbdev.showdoc import *

In [ ]:
#| export

from collections import defaultdict
from collections.abc import Sequence
from typing import Optional, Union
from copy import deepcopy

import numpy as np

import torch

from diffpass.constants import DEFAULT_AA_TO_INT

# Type aliases
SeqRecord = tuple[str, str]
SeqRecords = list[SeqRecord]
GroupwiseSeqRecords = dict[str, SeqRecords]

## Type aliases

```python
SeqRecord: tuple[str, str]
SeqRecords: list[SeqRecord]
GroupwiseSeqRecords: dict[str, SeqRecords]
```

In [ ]:
#| export

def create_groupwise_seq_records(
    seq_records: SeqRecords,
    group_name_func: callable,
    *,
    remove_groups_with_one_seq: bool = True,
) -> GroupwiseSeqRecords:
    """Group records of the form ``(header, sequence)`` in a collection by group name
    (e.g. species name), extracted from header information using `group_name_func`."""
    data_group_by_group = defaultdict(SeqRecords)
    for rec in seq_records:
        group_name = group_name_func(rec[0])
        data_group_by_group[group_name].append(rec)

    if remove_groups_with_one_seq:
        for group_name in list(data_group_by_group.keys()):
            if len(data_group_by_group[group_name]) < 2:
                data_group_by_group.pop(group_name)

    return data_group_by_group


def remove_groups_not_in_both(
    data_group_by_group_x: GroupwiseSeqRecords,
    data_group_by_group_y: GroupwiseSeqRecords,
) -> tuple[GroupwiseSeqRecords, GroupwiseSeqRecords]:
    """Remove groups that are not present in both input collections."""
    group_names = set(data_group_by_group_x.keys()) & set(data_group_by_group_y.keys())

    data_group_by_group_x_common = {
        group_name: data_group_by_group_x[group_name] for group_name in group_names
    }
    data_group_by_group_y_common = {
        group_name: data_group_by_group_y[group_name] for group_name in group_names
    }

    return data_group_by_group_x_common, data_group_by_group_y_common


def pad_msas_with_dummy_sequences(
    data_group_by_group_x: GroupwiseSeqRecords,
    data_group_by_group_y: GroupwiseSeqRecords,
    *,
    dummy_symbol: str = "-",
) -> tuple[GroupwiseSeqRecords, GroupwiseSeqRecords]:
    """Pad MSAs with dummy sequences so that all groups/species contain the same
    number of sequences."""
    # Check that all sequences in the x and y MSAs have the same length
    lengths_x = set(
        [
            len(seq)
            for data_x_this_group in data_group_by_group_x.values()
            for _, seq in data_x_this_group
        ]
    )
    lengths_y = set(
        [
            len(seq)
            for data_y_this_group in data_group_by_group_y.values()
            for _, seq in data_y_this_group
        ]
    )
    if len(lengths_x) != 1:
        raise ValueError(
            "Sequences in the first input collection must have the same lengths for padding with dummy gap sequences."
        )
    if len(lengths_y) != 1:
        raise ValueError(
            "Sequences in the second input collection must have the same lengths for padding with dummy gap sequences."
        )
    len_x = next(iter(lengths_x))
    len_y = next(iter(lengths_y))

    group_names = set(data_group_by_group_x.keys()) | set(data_group_by_group_y.keys())

    data_group_by_group_x_padded = defaultdict(SeqRecords)
    data_group_by_group_y_padded = defaultdict(SeqRecords)
    data_group_by_group_x_padded.update(deepcopy(data_group_by_group_x))
    data_group_by_group_y_padded.update(deepcopy(data_group_by_group_y))
    for group_name in group_names:
        max_depth = max(
            len(data_group_by_group_x[group_name]),
            len(data_group_by_group_y[group_name]),
        )
        data_group_by_group_x_padded[group_name] += [
            (f"dummy_{i}", dummy_symbol * len_x)
            for i in range(max_depth - len(data_group_by_group_x[group_name]))
        ]
        data_group_by_group_y_padded[group_name] += [
            (f"dummy_{i}", dummy_symbol * len_y)
            for i in range(max_depth - len(data_group_by_group_y[group_name]))
        ]

    return data_group_by_group_x_padded, data_group_by_group_y_padded


def get_single_and_paired_seqs(
    data_group_by_group_x: GroupwiseSeqRecords,
    data_group_by_group_y: GroupwiseSeqRecords,
    *,
    group_names: Optional[Sequence[str]] = None,
) -> dict[str, Union[list[list[tuple]], list[dict[str, int]]]]:
    """Single and paired sequences from two sequence records. The paired sequences are returned as a list of
    dictionaries, where the keys are the concatenated sequences and the values are the number of
    times that pair appears in the concatenated MSA."""
    if group_names is None:
        group_names = set(data_group_by_group_x.keys()) | set(
            data_group_by_group_y.keys()
        )

    x_seqs_by_group = {}
    y_seqs_by_group = {}
    xy_seqs_to_counts_by_group = {}
    for group_name in group_names:
        x_seqs_this_group = list(zip(*data_group_by_group_x[group_name]))[1]
        y_seqs_this_group = list(zip(*data_group_by_group_y[group_name]))[1]
        x_seqs_by_group[group_name] = x_seqs_this_group
        y_seqs_by_group[group_name] = y_seqs_this_group
        xy_seqs_to_counts_by_group[group_name] = {}
        xy_seqs_this_group = [
            f"{x_seq}:{y_seq}"
            for x_seq, y_seq in zip(x_seqs_this_group, y_seqs_this_group)
        ]
        if xy_seqs_this_group:
            unique_xy_this_group, counts_xy_this_group = np.unique(
                np.array(xy_seqs_this_group), return_counts=True
            )
            xy_seqs_to_counts_by_group[group_name].update(
                dict(zip(unique_xy_this_group, counts_xy_this_group))
            )

    return {
        "x_seqs_by_group": x_seqs_by_group,
        "y_seqs_by_group": y_seqs_by_group,
        "xy_seqs_to_counts_by_group": xy_seqs_to_counts_by_group,
    }

In [ ]:
show_doc(create_groupwise_seq_records)

---

[source](https://github.com/Bitbol-Lab/DiffPaSS/blob/main/diffpass/data_utils.py#L26){target="_blank" style="float:right; font-size:smaller"}

### create_groupwise_seq_records

>      create_groupwise_seq_records (seq_records:list[tuple[str,str]],
>                                    group_name_func:<built-infunctioncallable>,
>                                    remove_groups_with_one_seq:bool=True)

*Group records of the form ``(header, sequence)`` in a collection by group name
(e.g. species name), extracted from header information using `group_name_func`.*

In [ ]:
show_doc(remove_groups_not_in_both)

---

### remove_groups_not_in_both

>      remove_groups_not_in_both
>                                 (data_group_by_group_x:dict[str,list[tuple[str
>                                 ,str]]], data_group_by_group_y:dict[str,list[t
>                                 uple[str,str]]])

*Remove groups that are not present in both input collections.*

In [ ]:
show_doc(pad_msas_with_dummy_sequences)

---

[source](https://github.com/Bitbol-Lab/DiffPaSS/blob/main/diffpass/data_utils.py#L45){target="_blank" style="float:right; font-size:smaller"}

### pad_msas_with_dummy_sequences

>      pad_msas_with_dummy_sequences
>                                     (data_group_by_group_x:dict[str,list[tuple
>                                     [str,str]]], data_group_by_group_y:dict[st
>                                     r,list[tuple[str,str]]],
>                                     dummy_symbol:str='-')

*Pad MSAs with dummy sequences so that all groups/species contain the same
number of sequences.*

In [ ]:
#| export

def one_hot_encode_msa(
    seq_records: SeqRecords,
    aa_to_int: Optional[dict[str, int]] = None,
    device: Optional[torch.device] = None,
) -> torch.Tensor:
    """
    Given a list of records of the form (header, sequence), assumed to be a parsed MSA,
    tokenize each sequence and one-hot encode each token. Return a 3D tensor representing the
    one-hot encoded MSA.
    """
    if aa_to_int is None:
        aa_to_int = DEFAULT_AA_TO_INT

    tokenized_records = []
    for header, seq in seq_records:
        tokenized_records.append([aa_to_int[c] for c in seq])
    tokenized_records = torch.tensor(tokenized_records, device=device)

    tokenized_records_oh = torch.nn.functional.one_hot(tokenized_records).to(
        torch.get_default_dtype()
    )

    return tokenized_records_oh

In [ ]:
show_doc(one_hot_encode_msa)

---

[source](https://github.com/Bitbol-Lab/DiffPaSS/blob/main/diffpass/data_utils.py#L100){target="_blank" style="float:right; font-size:smaller"}

### one_hot_encode_msa

>      one_hot_encode_msa (seq_records:list[tuple[str,str]],
>                          aa_to_int:Optional[dict[str,int]]=None,
>                          device:Optional[torch.device]=None)

Given a list of records of the form (header, sequence), assumed to be a parsed MSA,
tokenize each sequence and one-hot encode each token. Return a 3D tensor representing the
one-hot encoded MSA.

In [ ]:
#| export

def compute_num_correct_pairings(
    hard_perms_by_group: list[np.ndarray],
    *,
    compare_to_identity_permutation: bool,
    single_and_paired_seqs: Optional[dict[str, list]] = None,
) -> int:
    """Compute the total number of correct pairings.
    'Correct' means that they are present in the original paired MSAs, assumed to be the
    ground truth.

    If `compare_to_identity_permutation` is True, then the correct pairings are assumed
    to be given by the identity permutation, and the `x_seqs`, `y_seqs`, and `xy_seqs`
    arguments are ignored.
    """
    correct = 0
    if compare_to_identity_permutation:
        for perm_this_group in hard_perms_by_group:
            n_seqs_this_group = perm_this_group.shape[-1]
            correct_this_group = np.sum(perm_this_group == np.arange(n_seqs_this_group))
            correct += correct_this_group
    else:
        x_seqs_by_group = single_and_paired_seqs["x_seqs_by_group"]
        y_seqs_by_group = single_and_paired_seqs["y_seqs_by_group"]
        xy_seqs_to_counts_by_group = single_and_paired_seqs[
            "xy_seqs_to_counts_by_group"
        ]
        for (
            perm_this_group,
            x_seqs_this_group,
            y_seqs_this_group,
            xy_seqs_to_counts_this_group,
        ) in zip(
            hard_perms_by_group,
            x_seqs_by_group,
            y_seqs_by_group,
            xy_seqs_to_counts_by_group,
        ):
            _xy_seqs_to_counts_this_group = xy_seqs_to_counts_this_group.copy()
            x_seqs_this_group_perm = [x_seqs_this_group[idx] for idx in perm_this_group]
            for x_seq, y_seq in zip(x_seqs_this_group_perm, y_seqs_this_group):
                xy_key = f"{x_seq}:{y_seq}"
                if _xy_seqs_to_counts_this_group.get(xy_key, 0) > 0:
                    _xy_seqs_to_counts_this_group[xy_key] -= 1
                    correct += 1

    return correct

In [ ]:
show_doc(compute_num_correct_pairings)

---

[source](https://github.com/Bitbol-Lab/DiffPaSS/blob/main/diffpass/data_utils.py#L125){target="_blank" style="float:right; font-size:smaller"}

### compute_num_correct_pairings

>      compute_num_correct_pairings (hard_perms_by_group:list[numpy.ndarray],
>                                    compare_to_identity_permutation:bool, singl
>                                    e_and_paired_seqs:Optional[dict[str,list]]=
>                                    None)

Compute the total number of correct pairings.
'Correct' means that they are present in the original paired MSAs, assumed to be the
ground truth.

If `compare_to_identity_permutation` is True, then the correct pairings are assumed
to be given by the identity permutation, and the `x_seqs`, `y_seqs`, and `xy_seqs`
arguments are ignored.

In [ ]:
#| export

def compute_comparable_group_idxs(
    group_sizes_arr: np.ndarray, *, max_size_ratio: int, max_group_size: int
) -> np.ndarray:
    # 1) Groups which have non-zero count for both families but where not both counts are equal to 1
    # NOTE: In AlphaFold this excludes the query (dummy group of size 1)
    mask_not_zero_in_one = (group_sizes_arr[:, 0] * group_sizes_arr[:, 1] != 0) * (
        group_sizes_arr[:, 0] * group_sizes_arr[:, 1] != 1
    )

    # 2) Groups which have non-zero count for both families and where not both counts are equal to 1 and where group size are not too different
    mask_comparable = (
        np.divide(
            np.max(group_sizes_arr, axis=1), np.min(group_sizes_arr, axis=1) + 1e-50
        )
        <= max_size_ratio
    )
    mask_comparable *= mask_not_zero_in_one

    # 3) Remove groups that are too large
    mask_max_group_size = np.max(group_sizes_arr, axis=1) <= max_group_size

    idx_comparable = np.flatnonzero(
        np.logical_and(mask_comparable, mask_max_group_size)
    )

    msa_depth = np.max(group_sizes_arr[idx_comparable], axis=1).sum()
    print(f"Final MSA_depth = {msa_depth}")

    print(f"Number of comparable groups = {len(idx_comparable)}")

    return idx_comparable